In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import einsum
from einops import rearrange
from torch.utils.data import DataLoader, Dataset, RandomSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.functional import mean_squared_error, mean_absolute_error

from transformers import BertModel, BertTokenizer

import numpy as np
import pandas as pd
from tqdm import tqdm
from tdc.multi_pred import DTI
from sklearn.metrics import average_precision_score

davis = DTI(name="davis")
davis.convert_to_log(form = 'binding')
davis_split = davis.get_split()

train_df = davis_split['train']
valid_df = davis_split['valid']
test_df = davis_split['test']

Found local copy...
Loading...
Done!
To log space...


In [2]:
class DTIDataset(Dataset):
    def __init__(self, data, molecule_tokenizer, protein_tokenizer):
        self.data = data
        
        self.molecule_max_len = 100
        self.protein_max_len = 512
        
        self.molecule_tokenizer = molecule_tokenizer
        self.protein_tokenizer = protein_tokenizer
    
        
    def molecule_encode(self, molecule_sequence):
        molecule_sequence = self.molecule_tokenizer(
            " ".join(molecule_sequence), 
            max_length=self.molecule_max_len, 
            truncation=True
        )
        
        return molecule_sequence
    
    
    def protein_encode(self, protein_sequence):
        protein_sequence = self.protein_tokenizer(
            " ".join(protein_sequence), 
            max_length=self.protein_max_len, 
            truncation=True
        )
        
        return protein_sequence
        
        
    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, idx):
        molecule_sequence = self.molecule_encode(self.data.loc[idx, "Drug"])
        protein_sequence = self.protein_encode(self.data.loc[idx, "Target"])
        y = self.data.loc[idx, "Y"]
                
        return molecule_sequence, protein_sequence, y


def collate_batch(batch):
    molecule_seq, protein_seq, y = [], [], []
    
    for (molecule_seq_, protein_seq_, y_) in batch:
        molecule_seq.append(molecule_seq_)
        protein_seq.append(protein_seq_)
        y.append(y_)
        
    molecule_seq = molecule_tokenizer.pad(molecule_seq, return_tensors="pt")
    protein_seq = protein_tokenizer.pad(protein_seq, return_tensors="pt")
    y = torch.tensor(y).float()
    
    return molecule_seq, protein_seq, y


molecule_tokenizer = molecule_tokenizer = BertTokenizer.from_pretrained("data/drug/molecule_tokenizer", model_max_length=128)
protein_tokenizer = BertTokenizer.from_pretrained("data/target/protein_tokenizer", do_lower_case=False)

train_dataset = DTIDataset(train_df, molecule_tokenizer, protein_tokenizer)
valid_dataset = DTIDataset(valid_df, molecule_tokenizer, protein_tokenizer)
test_dataset = DTIDataset(test_df, molecule_tokenizer, protein_tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=30, num_workers=8, 
                              pin_memory=True, prefetch_factor=10, 
                              drop_last=True, collate_fn=collate_batch, shuffle=True)

valid_dataloader = DataLoader(valid_dataset, batch_size=30, num_workers=8, 
                              pin_memory=True, prefetch_factor=10, 
                              collate_fn=collate_batch)

test_dataloader = DataLoader(test_dataset, batch_size=30, num_workers=8, 
                             pin_memory=True, prefetch_factor=10, 
                             collate_fn=collate_batch)

In [3]:
class DualLanguageModelDTI(nn.Module):
    def __init__(self, 
                 molecule_encoder, protein_encoder, hidden_dim=512,
                 molecule_input_dim=128, protein_input_dim=1024):
        super().__init__()
        self.molecule_encoder = molecule_encoder
        self.protein_encoder = protein_encoder
        
        # model freezing without last layer
        for param in self.molecule_encoder.encoder.layer[0:-1].parameters():
            param.requires_grad = False        
        for param in self.protein_encoder.encoder.layer[0:-1].parameters():
            param.requires_grad = False
        
        self.molecule_align = nn.Sequential(
            nn.LayerNorm(molecule_input_dim),
            nn.Linear(molecule_input_dim, hidden_dim)
        )
        
        self.protein_align = nn.Sequential(
            nn.LayerNorm(protein_input_dim),
            nn.Linear(protein_input_dim, hidden_dim)
        )
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.fc2 = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.fc3 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc_out = nn.Linear(hidden_dim, 1)
        
    
    def forward(self, molecule_seq, protein_seq):
        encoded_molecule = self.molecule_encoder(**molecule_seq)
        encoded_protein = self.protein_encoder(**protein_seq)
        
        cls_molecule = encoded_molecule.pooler_output
        cls_protein = encoded_protein.pooler_output
        
        cls_molecule = self.molecule_align(cls_molecule)
        cls_protein = self.protein_align(cls_protein)
        
        cls_concat = torch.cat([cls_molecule, cls_protein], dim=1)

        x = F.gelu(self.fc1(cls_concat))
        x = F.gelu(self.fc2(x))
        x = F.gelu(self.fc3(x))
        out = self.fc_out(x)
        
        return out

molecule_bert = BertModel.from_pretrained("weights/molecule_bert")
protein_bert = BertModel.from_pretrained("weights/protein_bert")
dlm_dti = DualLanguageModelDTI(molecule_bert, protein_bert)

Some weights of BertModel were not initialized from the model checkpoint at weights/molecule_bert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at weights/protein_bert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
class DTI_prediction(pl.LightningModule):
    def __init__(self, dlm_dti):
        super().__init__()
        self.model = dlm_dti

        
    def forward(self, molecule_sequence, protein_sequence):
        return self.model(molecule_sequence, protein_sequence)
    
    
    def training_step(self, batch, batch_idx):
        molecule_sequence, protein_sequence, y = batch
        
        y_hat = self(molecule_sequence, protein_sequence).squeeze(-1)        
        loss = F.mse_loss(y_hat, y)
        
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_mae", mean_absolute_error(y_hat, y), on_step=False, on_epoch=True, prog_bar=True)
        
        return loss
    
    
    def validation_step(self, batch, batch_idx):
        molecule_sequence, protein_sequence, y = batch
        
        y_hat = self(molecule_sequence, protein_sequence).squeeze(-1)        
        loss = F.mse_loss(y_hat, y)
        
        self.log('valid_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("valid_mae", mean_absolute_error(y_hat, y), on_step=False, on_epoch=True, prog_bar=True)
    
    
    def test_step(self, batch, batch_idx):
        molecule_sequence, protein_sequence, y = batch
        
        y_hat = self(molecule_sequence, protein_sequence).squeeze(-1)        
        loss = F.mse_loss(y_hat, y)
        
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_mae", mean_absolute_error(y_hat, y), on_step=False, on_epoch=True, prog_bar=True)
    
    
    def predict_step(self, batch, batch_idx):
        molecule_sequence, protein_sequence, y = batch
        
        y_hat = self(molecule_sequence, protein_sequence).squeeze(-1)        
        
        return y_hat

    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
        
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

    
    
callbacks = [
    ModelCheckpoint(monitor='valid_loss', save_top_k=3, dirpath='weights/dlm_dti_davis_512', filename='dlm_dti-{epoch:03d}-{valid_loss:.4f}-{valid_mae:.4f}'),
]

model = DTI_prediction(dlm_dti)
trainer = pl.Trainer(max_epochs=300, gpus=[0], enable_progress_bar=True, callbacks=callbacks)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# trainer.fit(model, train_dataloader, valid_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type                 | Params
-----------------------------------------------
0 | model | DualLanguageModelDTI | 424 M 
-----------------------------------------------
58.1 M    Trainable params
366 M     Non-trainable params
424 M     Total params
1,699.108 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [5]:
ckpt_fname = "dlm_dti-epoch=194-valid_loss=0.2036-valid_mae=0.2368.ckpt"

model = model.load_from_checkpoint("weights/dlm_dti_davis_512/" + ckpt_fname, dlm_dti=dlm_dti)
trainer.test(model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.19325804710388184
        test_mae            0.23516544699668884
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.19325804710388184, 'test_mae': 0.23516544699668884}]

In [6]:
pred = trainer.predict(model, test_dataloader)

true_ = test_dataloader.dataset.data.Y
pred_ = []

for i in pred:
    for j in i.tolist():
        pred_.append(j)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [7]:
def get_cindex(Y, P):
    summ = 0
    pair = 0
    
    for i in tqdm(range(1, len(Y))):
        for j in range(0, i):
            if i is not j:
                if(Y[i] > Y[j]):
                    pair +=1
                    summ +=  1* (P[i] > P[j]) + 0.5 * (P[i] == P[j])
        
            
    if pair is not 0:
        return summ/pair
    else:
        return 0


def r_squared_error(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    y_pred_mean = [np.mean(y_pred) for y in y_pred]

    mult = sum((y_pred - y_pred_mean) * (y_obs - y_obs_mean))
    mult = mult * mult

    y_obs_sq = sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))
    y_pred_sq = sum((y_pred - y_pred_mean) * (y_pred - y_pred_mean) )

    return mult / float(y_obs_sq * y_pred_sq)


def get_k(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)

    return sum(y_obs*y_pred) / float(sum(y_pred*y_pred))


def squared_error_zero(y_obs,y_pred):
    k = get_k(y_obs,y_pred)

    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    upp = sum((y_obs - (k*y_pred)) * (y_obs - (k* y_pred)))
    down= sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))

    return 1 - (upp / float(down))


def get_rm2(ys_orig, ys_line):
    r2 = r_squared_error(ys_orig, ys_line)
    r02 = squared_error_zero(ys_orig, ys_line)

    return r2 * (1 - np.sqrt(np.absolute((r2*r2)-(r02*r02))))

In [8]:
round(get_cindex(true_, pred_), 4)

100%|███████████████████████████████████████| 5153/5153 [00:58<00:00, 88.79it/s]


0.89

In [9]:
round(get_rm2(true_, pred_), 4)

0.6959

In [10]:
true_label = [1 if t >= 7 else 0 for t in true_]

round(average_precision_score(true_label, pred_), 4)

0.7643